In [ ]:
from pathlib import Path
import ansys.fluent.core as pyfluent


import os
import psutil

In [2]:
import ansys.fluent.core.datamodel.meshing as meshing

ModuleNotFoundError: No module named 'ansys.fluent.core.datamodel'

In [ ]:
session = pyfluent.launch_fluent(mode="meshing",processor_count=2, precision="double")


In [4]:
workflow =  session.workflow
workflow.InitializeWorkflow(WorkflowType="Watertight Geometry")

True

In [5]:
from inputs import *

In [6]:
workflow.TaskObject['Import Geometry'].Arguments = {
            'FileFormat': 'CAD', #[CAD, Mesh]
            'ImportType': 'Single File', # ['Single File', 'Multiple Files']
            'LengthUnit': 'mm', # ['mm', 'm', 'cm', 'in', 'ft', 'um', 'nm'] 
            'UseBodyLabels': 'No', # ['No', 'Yes']
            'FileName': GEOM_FILE_PATH
        }

In [ ]:
print(FACESIZES)

In [7]:
add_local_sizing = workflow.TaskObject["Add Local Sizing"]

for facesize in FACESIZES:
    add_local_sizing.Arguments = dict(
        {
            "AddChild": "yes",
            "SizeControlType": "FaceSize",
            "Name": facesize["Name"],
            "GrowthRate": facesize["GrowthRate"],
            "TargetMeshSize":facesize["TargetMeshSize"], #mm
            "FaceLabelList": facesize["FaceLabelList"],
        }
        )
    try:
        add_local_sizing.Execute()
        print(f"local sizing added.\nFacesize: {facesize}")
    except:
        print("error")



Importing one geom object per program-controlled and one zone per body ...
    D:\\ANSYS_V241\\ANSYS Inc\\v241\\commonfiles\\CPython\\3_10\\winx64\\Release\\python\\..\\Ansys\\TGrid\\CADReaders.py started by madan on DESKTOP-IR7JCSB winx64 on Fri Dec  6 19:57:39 2024
    using Python 3.10.9 (remotes/origin/4e018266dafe29ec2836492d71d245fcfd366d07-dirty:4e018266, Jan  3 2) [MSC v.1920 64 bit (AMD64)]
    
    using Ansys.Meshing.FieldMesher build Nov 27 2023 11:54:13
    
    running ANSYS TGrid CADToTGridConverter ...
    setting up parameters ...
    setting up parameters done.
    running conversion ...
    converting 1 file(s) from Workbench to FLTG using output path 'd:\\CFD_tui\\lemons-kitchen\\meshing\\watertight_workflow\\FM_DESKTOP-IR7JCSB_53200/out173347544953200.tgf'
    converting file 'enclosures.scdoc' (1 of 1) from Workbench to FLTG using output path 'd:\\CFD_tui\\lemons-kitchen\\meshing\\watertight_workflow\\FM_DESKTOP-IR7JCSB_53200'
    importing data ...
    importing

local sizing added.
Facesize: {'Name': 'facesize_blades', 'GrowthRate': 1.15, 'TargetMeshSize': -1, 'FaceLabelList': ['blades']}


In [9]:
generate_surface_mesh = workflow.TaskObject["Generate the Surface Mesh"]
generate_surface_mesh.Arguments = dict(
    {
        "CFDSurfaceMeshControls": {
            "CurvatureNormalAngle": SURFACE_MESH_PARAMS["CurvatureNormalAngle"], 
            "GrowthRate": SURFACE_MESH_PARAMS["GrowthRate"],
            "MaxSize": SURFACE_MESH_PARAMS["MaxSize"],
            "MinSize": SURFACE_MESH_PARAMS[ "MinSize"],
            "SizeFunctions": SURFACE_MESH_PARAMS["SizeFunctions"],
        }
    }
)
generate_surface_mesh.Execute()

generate_surface_mesh.InsertNextTask(CommandName="ImproveSurfaceMesh")
improve_surface_mesh = workflow.TaskObject["Improve Surface Mesh"]
improve_surface_mesh.Arguments.update_dict({"FaceQualityLimit": MESH_FACE_QUALITY_LIMIT})
improve_surface_mesh.Execute()


RuntimeError: Unknown exception

In [ ]:
describeTask=workflow.TaskObject["Describe Geometry"]
describeTask.Arguments = dict(
    CappingRequired="No",
    SetupType="The geometry consists of only fluid regions with no voids",
    applyShareTopology="Yes"
)
describeTask.Execute()

In [ ]:
describeTask=workflow.TaskObject["Describe Geometry"]
print(describeTask.TaskList)

In [ ]:
workflow.TaskObject["Create Regions"].Execute()


In [ ]:
updateBoundaries = workflow.TaskObject["Update Boundaries"]


In [ ]:
updateRegions = workflow.TaskObject["Update Regions"]